# OMMX Solver Adapter実装ガイド

本ドキュメントは、OMMXからSolver/Samplerを実行するためのAdapter（以下、OMMX Solver/Sampler Adapterと呼ぶ）を実装する際の推奨事項を記載したものです。下記の内容に従いOMMX Solver/Sampler Adapterを実装することで、他のOMMX Solver/Sampler Adapterと一定程度の一貫性を保つことができ、複数のOMMX Solver/Sampler Adapterを利用するユーザーにとってよりよいユーザー体験の提供に繋がります。そのため、OMMXコミュニティとしては以下の内容に従うことを推奨しています。

## OMMX Solver Adapterの推奨事項

- OMMXからSolverを利用するためのアダプター `OMMXxxxAdapter`は、抽象基底クラス `SolverAdapter`を継承して実装することを推奨する
    ```python
    from ommx.ommx_adapter import SolverAdapter

    class OMMXxxxAdapter(SolverAdapter):
        ...
    ``` 
- `OMMXxxxAdapter` のコンストラクタは、 `ommx.v1.Instance` を受け取り、以下のような処理を行うべきである
    - `ommx.v1.Instance` からSolverの入力形式（データやオブジェクト）を生成する処理
    - `OMMXxxxAdapter.decode` に必要となるデータの生成する処理
    ```python
    class OMMXxxxAdapter(SolverAdapter):
        def __init__(self, ommx_instance: ommx.v1.Instance):
            ...
    ```
- `OMMXxxxAdapter.solve` メソッドは、OMMXからSolverを簡単に使うためのクラスメソッドとして提供されるべきである
    - Solverのパラメーターを全て引数として実装する必要はない。そのようなユースケースは `OMMXxxxAdapter` クラスを直接使うことで実現できれば十分である
    ```python
    class OMMXxxxAdapter(SolverAdapter):
        ...
        @classmethod
        def solve(ommx_instance: ommx.v1.Instance) -> ommx.v1.Solution:
            ...
    ```
- `OMMXxxxAdapter.solver_input` プロパティは、OMMXからSolverの入力形式（データやオブジェクト）を取得するプロパティとして提供されるべきである
    ```python
    class OMMXxxxAdapter(SolverAdapter):
        ...
        @property
        def solver_input(self) -> SolverInput:
            ...
    ```
- `OMMXxxxAdapter.decode` メソッドは、Solverの出力形式（データやオブジェクト）を `ommx.v1.Solution` へと変換するメソッドとして提供されるべきである
    ```python
    class OMMXxxxAdpter(SolverAdapter):
        ...
        def decode(self, data: SolverOutput) -> ommx.v1.Solution:
            ...
    ```
- `SolverAdapter`に実装されているメソッドの引数以外のものを引数として定義する場合は、キーワード限定引数を利用し、デフォルト値を設定するものとする
    ```python
    class OMMXxxxAdapter(SolverAdapter):
        ...
        def __init__(self, ommx_instance: ommx.v1.Instance, *, kwarg="default", ...):
            ...
    ```

## OMMX Sampler Adapterの推奨事項

- OMMXからSamplerを利用するためのアダプター `OMMXxxxAdapter`は、抽象基底クラス `SamplerAdapter`を継承して実装することを推奨する
    ```python
    from ommx.ommx_adapter import SamplerAdapter

    class OMMXxxxAdapter(SamplerAdapter):
        ...
    ``` 
- `OMMXxxxAdapter` のコンストラクタは、 `ommx.v1.Instance` を受け取り、以下のような処理を行うべきである
    - `ommx.v1.Instance` からSamplerの入力形式（データやオブジェクト）を生成する処理
    - `OMMXxxxAdapter.decode_to_sampleset` に必要となるデータの生成する処理
    ```python
    class OMMXxxxAdapter(SamplerAdapter):
        def __init__(self, ommx_instance: ommx.v1.Instance):
            ...
    ```
- `OMMXxxxAdapter.sample` メソッドは、OMMXからSamplerを簡単に使うためのクラスメソッドとして提供されるべきである
    - Samplerのパラメーターを全て引数として実装する必要はない。そのようなユースケースは `OMMXxxxAdapter` クラスを直接使うことで実現できれば十分である
    ```python
    class OMMXxxxAdapter(SamplerAdapter):
        ...
        @classmethod
        def sample(ommx_instance: ommx.v1.Instance) -> ommx.v1.SampleSet:
            ...
    ```
- `OMMXxxxAdapter.sampler_input` プロパティは、OMMXからSamplerの入力形式（データやオブジェクト）を取得するプロパティとして提供されるべきである
    ```python
    class OMMXxxxAdapter(SamplerAdapter):
        ...
        @property
        def sampler_input(self) -> SamplerInput:
            ...
    ```
- `OMMXxxxAdapter.decode_to_sampleset` メソッドは、Samplerの出力形式（データやオブジェクト）を `ommx.v1.SampleSet` へと変換するメソッドとして提供されるべきである
    ```python
    class OMMXxxxAdpter(SamplerAdapter):
        ...
        def decode_to_sampleset(self, data: SamplerOutput) -> ommx.v1.SampleSet:
            ...
    ```
- `SamplerAdapter`に実装されているメソッドの引数以外のものを引数として定義する場合は、キーワード限定引数を利用し、デフォルト値を設定するものとする
    ```python
    class OMMXxxxAdapter(SamplerAdapter):
        ...
        def __init__(self, ommx_instance: ommx.v1.Instance, *, kwarg="default", ...):
            ...
    ```

## 検討事項

以下では、推奨するほどではないが実装を検討すると良いものを列挙する。

- `OMMXxxxAdapter.decode_to_state` メソッド: Solverの出力形式（データやオブジェクト）を `ommx.v1.State` に変換するためのメソッド
    
    ```python
    from ommx.ommx_adapter import SolverAdapter
    
    class OMMXxxxAdapter(SolverAdapter):
    	...
    	def decode_to_state(self, data: SolverOutput) -> ommx.v1.State:
    		...
    ```
    
- `yyy_to_instance` 関数: Solverの入力形式を `ommx.v1.Instance` に変換する関数
    
    ```python
    def xxx_to_instance(data: Any) -> ommmx.v1.Instance:
    		...
    ```

## 推奨事項を満たすOMMX Solver Adapterが保証するユーザー体験

上記の推奨事項を満たすOMMX Solver Adapterは、以下の一貫したユーザー体験を与えることができる。

- Solverを気軽に利用したいユーザー（Solverの入出力方法やパラメーターを把握したくないユーザー）は、共通で提供されている静的メソッド `solve` で簡単にSolverを実行できる
    
    ```python
    from ommx_xxx_adapter import OMMXxxxAdapter
    
    ommx_solution = OMMXxxxAdapter.solve(ommx_instance)
    ```
    
- Solverをチューニングして利用したいユーザーは、直接 `OMMXxxxAdapter` クラスを使うことでSolverのパラメーター等を設定して実行できる

    ```python
    from ommx_xxx_adapter import OMMXxxxAdapter

    adapter = OMMXxxxAdapter(ommx_instance)
    solver_input = adapter.solver_input
    
    ... # この部分でSolverのチューニングを行ってから実行し、solver_outputを得る
    
    ommx_solution = adapter.decode(solver_output)
    ```